In [17]:
yoi = [2020]
toi = ['15RTN', '15RTP', '14RNS']
spath = f"C:/Users/mcecil/CGA/CDL/CDL_HLS_dataframe{yoi[0]}.csv"
url_file = f"C:/Users/mcecil/CGA/CDL/CDL_HLS_dataframe{yoi[0]}.csv"
index_file = f'C:/Users/mcecil/CGA/CDL/mask_index_{yoi[0]}.csv'
kml_file = 'C:/Users/mcecil/CGA/CDL/sentinel_tile_grid.kml'
geojson_file = 'C:/Users/mcecil/CGA/CDL/aoi.geojson'  ## chip file, lat-long
geojson_rpj_file = 'C:/Users/mcecil/CGA/CDL/aoi_rpj.geojson'
hdf_dir = 'C:/Users/mcecil/CGA/CDL/hdf/'
tiff_dir = 'C:/Users/mcecil/CGA/CDL/tif/'
mask_dir = 'C:/Users/mcecil/CGA/CDL/mask/'

tile_src_path = "C:/Users/mcecil/CGA/CDL/tile_src.csv"

In [ ]:

# Switch plotting backend
import matplotlib
matplotlib.use('agg')

# Import required modules
#import cartopy.crs as ccrs
import os
from datetime import datetime
import pandas as pd
import geopandas
import pyhdf
import rasterio
import rasterio.mask
import matplotlib.pyplot as pp
import nasa_hls
import numpy as np
import time
import fiona
from pathlib import Path
import time
import urllib.request as urlreq
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# Load the HLS tiles and place there coordinates into a numpy array for processing later
fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
#del tile_src
tile_src.head(5)

In [ ]:
# Load the HLS query csv for this year
qfn = open(url_file)
qtile = []
qyear = []
qmonth = []
qday = []
qdate = []
qurl = []
for i, line in enumerate(qfn):
    if (i == 0): # Skip header
        continue
    dummy = line.split(",")
    qtile.append(dummy[2])
    qdate.append(dummy[3])
    qurl.append(dummy[4])
    
    qday.append(int(dummy[3].split("-")[2]))
    qmonth.append(int(dummy[3].split("-")[1]))
    qyear.append(int(dummy[3].split("-")[0]))
qfn.close()
qdate = np.array(qdate)
qtile = np.array(qtile)
qurl = np.array(qurl)
qmonth = np.array(qmonth)
qyear = np.array(qyear)

In [ ]:
# Create the index file
# fn_index = open(index_file, "w")
# fn_index.write('EventID,Name,FireDate,MaskDate,BurnAcres,Tile,MaskFile')
# fn_index.close()

Load geojson used for query. 

In [ ]:
aoi_src = geopandas.read_file(geojson_file) ## this could be a list
nfeatures = aoi_src.shape[0]
aoi_src = pd.concat([aoi_src, aoi_src.bounds], axis = 1)
aoi_src['centroid_x'] = aoi_src.centroid.x
aoi_src['centroid_y'] = aoi_src.centroid.y
aoi_src.head(5)

Loop through aoi_src (chips), associate closest tile, add ID field, download hdf

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-bs_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):     
        local_name = cu.split('/')[-1].replace("\n", "")
        # Download a candidate file to check cloud cover
        try:
            urlreq.urlretrieve(cu, filename=hdf_dir+local_name)
        except:
            continue
#aoi_src.head(5)

Extract cloud cover and spatial coverage from metadata (NOT WORKING)

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-bs_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
       # print(cu)
        local_name = cu.split('/')[-1].replace("\n", "")
        
        if local_name != "HLS.S30.T14RNS.2020117.v1.4.hdf":
            continue
        # Download a candidate file to check cloud cover
        try:
            md = nasa_hls.get_metadata_from_hdf(hdf_dir+local_name, fields=['cloud_cover', 'spatial_coverage'])
            print(md)
       #     if ((md["cloud_cover"] < cldfrac_threshold) and (md['spatial_coverage'] > spacecov_threshold)):
        except:
            continue
        
aoi_src.head(5)

Convert HDF to single-layer TIF (Using HLS library, DOES NOT WORK)

In [19]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-bs_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
       # print(cu)
        local_name = cu.split('/')[-1].replace("\n", "")
        nasa_hls.convert_hdf2tiffs(Path(hdf_dir+local_name), Path(tiff_dir))

ERROR DURING CONVERSION WITH CMD: gdal_translate HDF4_EOS:EOS_GRID:'C:\Users\mcecil\CGA\CDL\hdf\HLS.S30.T14RNS.2020002.v1.4.hdf':Grid:B01 C:\Users\mcecil\CGA\CDL\tif\HLS.S30.T14RNS.2020002.v1.4\HLS.S30.T14RNS.2020002.v1.4__Coastal_Aerosol.tif.
Traceback (most recent call last):
  File "C:\Users\mcecil\AppData\Local\anaconda3\envs\IBM_CDL\lib\site-packages\nasa_hls\hdf2tiff_conversion.py", line 82, in convert_hdf2tiffs
    subprocess.check_call(cmd, shell=True)
  File "C:\Users\mcecil\AppData\Local\anaconda3\envs\IBM_CDL\lib\subprocess.py", line 369, in check_call
    raise CalledProcessError(retcode, cmd)
subprocess.CalledProcessError: Command 'gdal_translate HDF4_EOS:EOS_GRID:'C:\Users\mcecil\CGA\CDL\hdf\HLS.S30.T14RNS.2020002.v1.4.hdf':Grid:B01 C:\Users\mcecil\CGA\CDL\tif\HLS.S30.T14RNS.2020002.v1.4\HLS.S30.T14RNS.2020002.v1.4__Coastal_Aerosol.tif' returned non-zero exit status 1.


geometry      POLYGON ((-98.46590447567304 28.48216163356960...
minx                                                 -98.492844
miny                                                  28.482162
maxx                                                 -98.465904
maxy                                                  28.491892
centroid_x                                           -98.479374
centroid_y                                            28.487027
tile                                                      14RNS
chip_id                                                  chip_0
Name: 0, dtype: object
        minx       miny       maxx       maxy
0 -98.492844  28.482162 -98.465904  28.491892
[69873.57854445 26546.24690202 22692.05707804 ... 15908.73972509
 12442.80659328  9532.39628432]
14RNS


ERROR DURING CONVERSION WITH CMD: gdal_translate HDF4_EOS:EOS_GRID:'C:\Users\mcecil\CGA\CDL\hdf\HLS.S30.T14RNS.2020002.v1.4.hdf':Grid:B02 C:\Users\mcecil\CGA\CDL\tif\HLS.S30.T14RNS.2020002.v1.4\HLS.S30.T14RNS.2020002.v1.4__Blue.tif.
Traceback (most recent call last):
  File "C:\Users\mcecil\AppData\Local\anaconda3\envs\IBM_CDL\lib\site-packages\nasa_hls\hdf2tiff_conversion.py", line 82, in convert_hdf2tiffs
    subprocess.check_call(cmd, shell=True)
  File "C:\Users\mcecil\AppData\Local\anaconda3\envs\IBM_CDL\lib\subprocess.py", line 369, in check_call
    raise CalledProcessError(retcode, cmd)
subprocess.CalledProcessError: Command 'gdal_translate HDF4_EOS:EOS_GRID:'C:\Users\mcecil\CGA\CDL\hdf\HLS.S30.T14RNS.2020002.v1.4.hdf':Grid:B02 C:\Users\mcecil\CGA\CDL\tif\HLS.S30.T14RNS.2020002.v1.4\HLS.S30.T14RNS.2020002.v1.4__Blue.tif' returned non-zero exit status 1.
ERROR DURING CONVERSION WITH CMD: gdal_translate HDF4_EOS:EOS_GRID:'C:\Users\mcecil\CGA\CDL\hdf\HLS.S30.T14RNS.2020002.v1.4.hd

KeyboardInterrupt: 

Convert HDF to single-layer TIF (Mike's way - works)

In [18]:
from osgeo import gdal
import glob
from subprocess import Popen, PIPE

bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12', 'QA']


hdf_files = glob.glob(hdf_dir + "*.hdf")
for hdf_file in hdf_files:
    print(hdf_file)
    file_name = hdf_file.split('\\')[-1].replace('.hdf', '')
    for long_band_name in bands:
        cmd = f'gdal_translate HDF4_EOS:EOS_GRID:"' + hdf_file + '":Grid:' + long_band_name + ' ' + tiff_dir + file_name + '__' + long_band_name + '.tif'
       # print(cmd)
        p = Popen(cmd, stdout=PIPE, shell=True)
       # print(p)
        output, err = p.communicate()
      #  print(output)
       # print(err)
  #  nasa_hls.convert_hdf2tif_dirfs(Path(hdf_file), Path(tif_dir_dir))
    # ds = gdal.Open(hdf_file)
    # ds_t = gdal.Translate('output.tif', hdf_file)
#ds = None
#print(ds)


C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020002.v1.4.hdf
C:/Users/mcecil/CGA/CDL/hdf\HLS.S30.T14RNS.2020005.v1.4.hdf


KeyboardInterrupt: 